In [ ]:
import json
from pathlib import Path

# new cell (index 0): load spider_data.json without overwriting if already defined below
try:
    spider_data  # if this exists, don't overwrite
except NameError:

    path = Path("spider_data_2.json")
    if not path.exists():
        raise FileNotFoundError("spider_data.json not found in the current working directory")

    with path.open("r", encoding="utf-8") as f:
        spider_data = json.load(f)

In [ ]:
import numpy as np

vertices = [item['vertices'] for item in spider_data]

min_vertices = min(vertices)
max_vertices = max(vertices)
mean_vertices = np.mean(vertices)

print(f"Minimum vertices: {min_vertices}")
print(f"Maximum vertices: {max_vertices}")
print(f"Mean vertices: {mean_vertices:.2f}")

In [ ]:
# Group vertices by ranges of 10
vertex_ranges = {}
seen_graphs = set()

for item in spider_data:
    gi = item['graph_index']
    if gi not in seen_graphs:
        v = item['vertices']
        range_key = (v // 10) * 10
        range_label = f"{range_key}-{range_key + 10}"
        vertex_ranges[range_label] = vertex_ranges.get(range_label, 0) + 1
        seen_graphs.add(gi)

# Sort ranges and plot
sorted_ranges = sorted(vertex_ranges.keys(), key=lambda x: int(x.split('-')[0]))
counts = [vertex_ranges[r] for r in sorted_ranges]

plt.figure(figsize=(4, 3))
plt.rcParams.update({'font.family': 'arial', 'font.size': 11})
plt.bar(sorted_ranges, counts, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Number of Vertices')
plt.ylabel('Number of Graphs')
plt.title('Distribution of Graph Sizes')
plt.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
epsilon_values = sorted(set(item['epsilon'] for item in spider_data))
print(epsilon_values)

In [ ]:
# Filter invalid adaptive upperbound values
filtered_spider_data = [item for item in spider_data if item['adaptive_upper_bound'] > 0]

import matplotlib.pyplot as plt

# Create a plot for each epsilon value
for epsilon in epsilon_values:
    epsilon_data = [item for item in filtered_spider_data if item['epsilon'] == epsilon]
    
    vertices_list = [item['vertices'] for item in epsilon_data]
    means = [np.mean(item['nonadaptive_runs']) / (item['adaptive_upper_bound'] * item['epsilon'] / 24) for item in epsilon_data]
    mins = [np.min(item['nonadaptive_runs']) / (item['adaptive_upper_bound'] * item['epsilon'] / 24) for item in epsilon_data]
    maxs = [np.max(item['nonadaptive_runs']) / (item['adaptive_upper_bound'] * item['epsilon'] / 24) for item in epsilon_data]
    
    for item in epsilon_data:
        if np.min(item['nonadaptive_runs']) / (item['adaptive_upper_bound'] * item['epsilon'] / 24) < 1:
            print(item['graph_index'] , ' ', np.min(item['nonadaptive_runs']))

    plt.figure(figsize=(4, 3))
    plt.rcParams.update({'font.family': 'arial', 'font.size': 11})
    plt.scatter(vertices_list, means, label='Mean', color='blue', alpha=0.6)
    plt.scatter(vertices_list, mins, label='Min', color='red', alpha=0.6)
    plt.scatter(vertices_list, maxs, label='Max', color='green', alpha=0.6)
    plt.axhline(y=1, color='black', linestyle='--', linewidth=2, label='Critical value')
    plt.xlabel('Number of Vertices')
    plt.ylabel('Ratio of performance and alpha')
    plt.title(f'Epsilon = {epsilon}')
    plt.yscale('log')
    plt.ylim(0.1, 10000)
    # plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
mean_rewards_per_budget = []
for epsilon in epsilon_values:
    per_graph = [
        np.mean(item['nonadaptive_runs']) / (1 + epsilon)
        for item in filtered_spider_data
        if item['epsilon'] == epsilon
    ]
    mean_rewards_per_budget.append(np.mean(per_graph))

plt.figure(figsize=(4, 3))
plt.rcParams.update({'font.family': 'arial', 'font.size': 11})
plt.plot(epsilon_values, mean_rewards_per_budget, marker='o', linestyle='-', color='blue')
plt.xlabel('Epsilon Value')
plt.ylabel('Mean Reward per Budget')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
mean_rewards_per_vertex = []
for epsilon in epsilon_values:
    per_graph = [
        np.mean(item['nonadaptive_runs']) / item['vertices']
        for item in filtered_spider_data
        if item['epsilon'] == epsilon
    ]
    mean_rewards_per_vertex.append(np.mean(per_graph))

plt.figure(figsize=(4, 3))
plt.rcParams.update({'font.family': 'arial', 'font.size': 11})
plt.plot(epsilon_values, mean_rewards_per_vertex, marker='o', linestyle='-', color='blue')
plt.xlabel('Epsilon Value')
plt.ylabel('Mean Reward per Vertex')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Aggregate nonadaptive runs by graph_index (regardless of epsilon)
grouped_runs = {}
legs_by_graph = {}
for item in filtered_spider_data:
    gi = item['graph_index']
    grouped_runs.setdefault(gi, []).extend(item['nonadaptive_runs'])
    if gi not in legs_by_graph:
        legs_by_graph[gi] = item['legs']

# Compute mean reward per graph_index
graph_indices = sorted(grouped_runs.keys())
mean_reward_per_graph = [np.mean(grouped_runs[gi]) for gi in graph_indices]
legs_per_graph = [legs_by_graph[gi] for gi in graph_indices]

# Prepare plot: scatter of each graph (legs vs mean reward)
plt.figure(figsize=(6, 4))
plt.rcParams.update({'font.family': 'arial', 'font.size': 11})
plt.scatter(legs_per_graph, mean_reward_per_graph, alpha=0.7, edgecolor='k')

# Also plot mean across graphs that share the same number of legs
unique_legs = sorted(set(legs_per_graph))
mean_by_legs = [
    np.mean([m for l, m in zip(legs_per_graph, mean_reward_per_graph) if l == ul])
    for ul in unique_legs
]
plt.plot(unique_legs, mean_by_legs, color='red', marker='o', linestyle='-', linewidth=2, label='Mean per legs')

plt.xlabel('Number of Legs')
plt.ylabel('Mean Nonadaptive Reward')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Aggregate nonadaptive runs by graph_index (regardless of epsilon)
grouped_runs = {}
depth_by_graph = {}
for item in filtered_spider_data:
    gi = item['graph_index']
    grouped_runs.setdefault(gi, []).extend(item['nonadaptive_runs'])
    if gi not in depth_by_graph:
        depth_by_graph[gi] = item['depth']

# Compute mean reward per graph_index
graph_indices = sorted(grouped_runs.keys())
mean_reward_per_graph = [np.mean(grouped_runs[gi]) for gi in graph_indices]
depth_per_graph = [depth_by_graph[gi] for gi in graph_indices]

# Prepare plot: scatter of each graph (depth vs mean reward)
plt.figure(figsize=(6, 4))
plt.rcParams.update({'font.family': 'arial', 'font.size': 11})
plt.scatter(depth_per_graph, mean_reward_per_graph, alpha=0.7, edgecolor='k')

# Also plot mean across graphs that share the same number of depth
unique_depth = sorted(set(depth_per_graph))
mean_by_depth = [
    np.mean([m for l, m in zip(depth_per_graph, mean_reward_per_graph) if l == ul])
    for ul in unique_depth
]
plt.plot(unique_depth, mean_by_depth, color='red', marker='o', linestyle='-', linewidth=2, label='Mean per Depth')

plt.xlabel('Depth')
plt.ylabel('Mean Nonadaptive Reward')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()